In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('../../../../')
print(os.getcwd())
from v2.strategy.indicators.optimal_v2 import Optimal_v2
from v2.model import Trading
from v2.strategy.indicators.param import Param
from v2.strategy.indicators.roc import RateOfChange
from v2.research.scripts.notebook_utils import notebookUtils
from v2.research.scripts.scoreboard import updateScoreboard
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from v2.strategy.strategies.atlas.atlas_v1_2 import Atlas_v1_2
from v2.strategy.strategies.atlas.atlas_v0_0 import Atlas_v0_0

/home/marshingjay/Repos/moonshot/back_testing


In [2]:
nu = notebookUtils()
model_name = "skunk"

In [3]:
def simulate_all_trades(dataset, strategy):
    dataset['is_potential_buy'] = False
    coin = 'COIN'
    generic_strategy = strategy([coin])
    opened_trades = dict()
    closed_trades = dict()
    for row in dataset.itertuples():
        otkeys = list(opened_trades.keys())
        for tick in otkeys:
            opened_trades[tick][0].process(row, coin)
            if opened_trades[tick][0].calc_exit(row, coin):
                closed_trades[tick] = (row.close/opened_trades[tick][1]) - 1
                del opened_trades[tick]
        if generic_strategy.calc_entry(row, coin):
            dataset.loc[(dataset.time == row.time),'is_potential_buy']=True
            opened_trades[row.time] = (generic_strategy, row.close)
            generic_strategy = strategy([coin])
        
    dataset['simul_profit'] = 0.0
    for k in closed_trades:
        dataset.loc[(dataset.time == k),'simul_profit']=closed_trades[k]

In [4]:
datasets, features, indicators = nu.loadData(indicators=[('SMA','close',''),('SMA','close','for_short'),('roc','SMA_for_short', 'shorter')],
                            param_spec={
                                'SMA':{'period':150}, 
                                'SMA_for_short':{'period':30}, 
                                'RateOfChange_shorter':{'period':45},
                                },
                            spans=[],
                            seperate_by_coin=True)



Loading data from ONE...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from ETC...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from chunk 5...
Loading data from chunk 6...
Loading data from chunk 7...
Loading data from ONT...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from chunk 5...
Loading data from chunk 6...
Loading data from chunk 7...
Loading data from BCH...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from RVN...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from EOS...
Loading data from chunk 0...
Loading d

In [5]:
datasets

9  0.2607  0.2589  0.2593  0.2596  100272.82  0.266169   
 594744  1593308999999  0.2602  0.2594  0.2597  0.2599   23572.19  0.266096   
 594745  1593309059999  0.2605  0.2602  0.2605  0.2602   27384.91  0.266025   
 594746  1593309119999  0.2615  0.2604  0.2610  0.2607   70990.92  0.265955   
 594747  1593309179999  0.2611  0.2607  0.2608  0.2608    7060.71  0.265885   
 594748  1593309239999  0.2611  0.2608  0.2608  0.2608    6755.86  0.265816   
 594749  1593309299999  0.2610  0.2603  0.2604  0.2607   15482.84  0.265742   
 594750  1593309359999  0.2609  0.2602  0.2605  0.2607     872.09  0.265667   
 594751  1593309419999  0.2604  0.2598  0.2598  0.2604   12991.86  0.265589   
 594752  1593309479999  0.2600  0.2594  0.2594  0.2598   16201.37  0.265509   
 594753  1593309539999  0.2597  0.2597  0.2597  0.2597     375.55  0.265436   
 594754  1593309599999  0.2607  0.2597  0.2605  0.2597   10263.66  0.265367   
 594755  1593322259999  0.2624  0.2605  0.2615  0.2605  134750.76  0.2653

In [6]:
features

['SMA', 'SMA_for_short', 'RateOfChange_shorter']

In [7]:
for d in datasets:
    simulate_all_trades(d, Atlas_v0_0)

KeyboardInterrupt: 

In [7]:
dataset = pd.concat(datasets)


In [8]:
buys = dataset[dataset['is_potential_buy'] == True]

In [9]:
sum(buys['simul_profit']) / len(buys['simul_profit'])

0.005250809897387273

In [49]:
model_dataset, features, indicators = nu.loadData(indicators=[('BOP', 'close', ''), ('PSAR', 'close', '')],
                            param_spec={},
                            
                            spans=[
                                    {'indicator_name':'NATR',
                                    'column_name': 'close',
                                    'param_name': 'period',
                                    'param_values': [300]},
                                    {'indicator_name':'CMO',
                                    'column_name': 'close',
                                    'param_name': 'period',
                                    'param_values': [10, 30, 200]},
                                    {'indicator_name':'CCI',
                                    'column_name': 'close',
                                    'param_name': 'period',
                                    'param_values': [30, 200]},
                                    {'indicator_name':'WILLR',
                                    'column_name': 'close',
                                    'param_name': 'period',
                                    'param_values': [30, 200]},
                                    
                                    ],

                            scale='minmaxwindow')

Loading data from ZEC...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from ZRX...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from ATOM...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from DASH...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from ONE...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from BAT...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading

In [50]:
joined_dataset = model_dataset.merge(dataset, on=['time', 'open', 'high', 'low', 'close', 'volume'])

In [51]:
joined_dataset = joined_dataset[joined_dataset['is_potential_buy'] == True]

In [52]:
joined_dataset.drop(["time", "open", "high", "volume", "low", "SMA", "SMA_for_short", "RateOfChange_shorter", "is_potential_buy"], axis=1, inplace=True)

joined_dataset.dropna(inplace=True)


In [53]:
joined_dataset["optimal"] = joined_dataset.apply(lambda x: nu.filter_optimal(x.simul_profit, (0.00525, 1.0), 'buy'),  axis=1)

In [54]:
joined_dataset.drop(["simul_profit"], axis=1, inplace=True)

In [55]:
joined_dataset

,close,BOP,PSAR,NATR_close_period_300,CMO_close_period_10,CMO_close_period_30,CMO_close_period_200,CCI_close_period_30,CCI_close_period_200,WILLR_close_period_30,WILLR_close_period_200,optimal
3898,47.960,0.197917,0.889387,0.181734,0.001526,0.024391,0.000000,0.331656,0.000000,4.301075e-02,3.809524e-02,0.0
3901,47.980,0.111111,0.748432,0.218538,0.150726,0.139315,0.054898,0.403216,0.089080,5.376344e-02,4.761905e-02,0.0
3902,47.900,0.250000,0.735690,0.226227,0.142062,0.132669,0.042235,0.412221,0.097920,1.075269e-02,9.523810e-03,0.0
3905,48.040,0.000000,0.535984,0.257385,0.279703,0.233379,0.112317,0.445251,0.192614,8.602151e-02,7.619048e-02,0.0
3906,48.060,0.150000,0.533613,0.271365,0.290613,0.240418,0.118221,0.449855,0.208465,9.677419e-02,8.571429e-02,0.0
3907,47.900,0.230769,0.687768,0.281262,0.256134,0.221962,0.093479,0.446263,0.198212,1.604278e-02,1.421801e-02,0.0
3908,47.720,0.227273,0.728038,0.298710,0.223040,0.203343,0.067167,0.443268,0.192855,5.607477e-02,5.042017e-02,0.0
3909,46.920,0.038462,0.910464,0.353495,0.136075,0.143556,0.000000,0.424363,0.140862,2.422145e-02,2.236422e-02,0.0
3910,47.110,0.837838,0.844215,0.369254,0.216651,0.195782,0.048398,0.423777,0.145739,1.233333e-01,1.141975e-01,0.0
3911,46.730,0.026316,0.905694,0.392114,0.179426,0.170976,0.008446,0.425361,0.157727,1.421085e-16,1.421085e-16,0.0


In [56]:
trainX, testX, trainy, testy = nu.splitData(joined_dataset, split_size=0.2, y_column_name="optimal", shuffle_data=False, balance_unbalanced_data=True, balance_info={'multiplier_val':1, 'superset_class_val':0, 'randomize_concat':True})

In [57]:
len(trainX)

61637

In [58]:
class_weights = nu.getWeights(trainy)

In [59]:
clf = RandomForestClassifier(random_state=69420, class_weight=class_weights, n_jobs=-1, n_estimators=100, criterion='entropy', min_samples_leaf=5)
clf.fit(trainX.drop("close", axis=1).values, trainy.values)
print(clf.score(testX.drop("close", axis=1).values, testy.values))

0.6075924724205062


In [60]:

feature_importances = permutation_importance(clf, trainX.drop("close", axis=1).values, trainy.values, n_repeats=10, random_state=69420)

for i in range(len(feature_importances.importances_mean)):
    print('i: {}, Feature: {}, Score: {}'.format(i, trainX.drop("close", axis=1).columns[i], feature_importances.importances_mean[i]))

i: 0, Feature: BOP, Score: 0.013147946850106274
i: 1, Feature: PSAR, Score: 0.08472183915505298
i: 2, Feature: NATR_close_period_300, Score: 0.21238704025179683
i: 3, Feature: CMO_close_period_10, Score: 0.09142073754400765
i: 4, Feature: CMO_close_period_30, Score: 0.10483800314746013
i: 5, Feature: CMO_close_period_200, Score: 0.22236805814689226
i: 6, Feature: CCI_close_period_30, Score: 0.142795723347989
i: 7, Feature: CCI_close_period_200, Score: 0.1627674935509515
i: 8, Feature: WILLR_close_period_30, Score: 0.06921654201210312
i: 9, Feature: WILLR_close_period_200, Score: 0.12353132047309245


In [62]:
# model_version = nu.exportModel(clf, 'playmaker', new_version=False, indicators=indicators, features=features, proba_threshold=0.5, is_nn=False)